## Program Flow

1. モジュールインポート


In [1]:
# scikit learnのテストデータの読み込み
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

# scikit learnの分類器であるSupport Vector Machineの読み込み
from sklearn.svm import LinearSVC as Classifier 

# scikit learnの精度測定ツールの読み込み
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 

# pandas(pd) とNumpy(np)のモジュール
import numpy as np
import pandas as pd
import sys,os,random
import warnings

ImportError: cannot import name 'fetch_mldata' from 'sklearn.datasets' (/Users/fujiccofujimoto/.pyenv/versions/Anaconda3-2019.10/lib/python3.7/site-packages/sklearn/datasets/__init__.py)

# 変数定義

In [ ]:
pram_traing_rate=0.8     #トレーニングデータとテストデータの比率

# 前処理済みファイルのロード

In [ ]:
# 乱数のシード固定
np.random.seed(0)
random.seed(0)
# データフレームの表示件数の設定
pd.set_option('display.max_columns', 100)

# データロード
df = pd.read_csv('data/fx_preprocessing.csv', index_col=None)

#df[df['Date']=='2018-12-31']
df.tail()
df.head()


y_loc_s=df.columns.get_loc('label_up')
y_loc_e=df.columns.get_loc('label_down')
df.tail(3)

In [ ]:
#トレーニングデータと正解ラベルに分割
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

# 前後のデータのカット
df3=df.loc[30:(len(df)-30)]
fx_len=len(df3)


# label数を揃える
min_labels=[df3[df3['label_up']==1].shape[0],
     df3[df3['label_none']==1].shape[0],
     df3[df3['label_down']==1].shape[0]]

min_sample=min(min_labels)


# サンプル後再度結合
df3=pd.concat([
    df3[df3['label_up']==1].sample(n=min_sample)
    ,df3[df3['label_down']==1].sample(n=min_sample)
    ,df3[df3['label_none']==1].sample(n=min_sample)
    ])

print(df3[df3['label_up']==1].shape)
print(df3[df3['label_none']==1].shape)
print(df3[df3['label_down']==1].shape)

In [ ]:
# データの分割
train, test = train_test_split(df3, train_size=pram_traing_rate,shuffle=False)

#トレーニングデータと正解ラベルに分割
trainX,testX=train.iloc[:,3:y_loc_s],test.iloc[:,3:y_loc_s]
trainY,testY=train.iloc[:,y_loc_s:y_loc_e+1],test.iloc[:,y_loc_s:y_loc_e+1]


#Numpy形式に変換
trainX,testX=trainX.as_matrix(),testX.as_matrix()
trainY,testY=trainY.as_matrix(),testY.as_matrix()

#データサンプル数の確認
print("トレーニングデータ",trainX.shape)
print("トレーニングデータ",trainY.shape)
print("　 　 テストデータ",testX.shape)
print("　 　 テストデータ",testY.shape)

# sckit-learnの準備


In [ ]:
testY2 =np.argmax(testY,axis=1)
trainY2=np.argmax(trainY,axis=1)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier 

In [ ]:
names = [
         "Random Forest", 
        "Logistic Regression", 
         "SVM",
         "Decision Tree", 
         "Neural Net", 
         "Naive Bayes",
         "AdaBoostClassifier"
        ]

# Initializing Classifiers
classifiers = [
    RandomForestClassifier(random_state=1, n_estimators=100),
    LogisticRegression(),
    SVC(kernel="linear"),
    DecisionTreeClassifier(),
    MLPClassifier(),
    GaussianNB(),
    AdaBoostClassifier()
]

In [ ]:
columns=['name','accuracy']
df_acc=pd.DataFrame(data=None,columns=columns)

for classifier, name in zip(classifiers, names):
    
    #　該当分類器にて、学習
    classifier.fit(trainX ,trainY2)
 
    #　分類器の正答率を求める
    pre=classifier.predict(testX)
    ac_score=accuracy_score(testY2,pre)
    print('分類器:{:} , 精度:{:.2%}'.format(name,ac_score))

    # 結果をデータフレームに入れる
    s=pd.Series(data=[name,ac_score],index=columns)
    df_acc=df_acc.append(s,ignore_index=True)

In [ ]:
%matplotlib inline
# データフレームの順番を分類器の精度順に表示
df_acc=df_acc.sort_values(by=['accuracy'],ascending=True)

# データフレームのラッパAPIからmatplotのapiをコールして、グラフを作成
df_acc.plot.barh(x='name',y='accuracy',legend=False)

In [ ]:
classifier=classifiers[
df_acc[df_acc['accuracy']==df_acc['accuracy'].max()].index[0]
]
classifier.fit(trainX ,trainY2)
 

# confusion_matrix

In [ ]:
from sklearn import metrics
print("0.label_up  ",test[test['label_up']==1].shape[0])
print("1.label_none",test[test['label_none']==1].shape[0])
print("2.label_down",test[test['label_down']==1].shape[0])

predict_y=classifier.predict(testX)
print("confusion matrix:")
print(metrics.confusion_matrix( testY2,predict_y))
print()
print("classification report:")
target_names = ['up','none','down']
print(metrics.classification_report(testY2,predict_y))

# シミュレーション

In [ ]:
close_index=df.columns.get_loc('Close')
open_index=df.columns.get_loc('Open')
date_index=df.columns.get_loc('Date')

MONEY=1000000
LEN=len(df2_num)

start=len(trainX)
end=len(df2_num)-1
print("start:",df2_num[start,0],"〜end:",df2_num[end,0])

standard=1000000/df2_num[start,1]  
DATE=""
result_out=""
df_sim = pd.DataFrame( columns=['Date', 'Open', 'Close', 'Up', 'money','基準価格','result','predict','正誤','利益率'] )

In [ ]:
for i in range(start,end): 
        PRE=df2_num[i,np.array([
                               3,4,5,
                               6,7,8,
                               9,10,11,
                               12,13,14,
                               15,16,17,
                               18,19,20,
                               21,22,23,
                               24,25,26,
                               27,28,29,
                               30,31,32,

                               33,34,35,
                               36,37,38,
                               39,40,41,
                               42,43,44,
                               45,46,47,
                               48,49,50,
                               51,52,53,
                               54,55,56,
                               57,58,59,
                               60,61,62,
                               ])]
        
        #予測のコマンド
        PRE2 = classifier.predict_proba(np.resize(PRE,(1,trainX.shape[1])))
        PRE2=PRE2[0]
        
        profit_rate=0
        DATE2=df2_num[i,date_index]
        if DATE != DATE2[:7]:       #月初の定点観測
            DATE=DATE2[:7]
            print()
            result_out=result_out+"\n"
            if DATE2[5:7]=="01":
                print("--------------------------------------------------------------------")
                result_out=result_out+"--------------------------------------------------------------------"+"\n"
            STRING=df2_num[i,0]+":資金"+"{:,}".format(int(MONEY))+"(基準"+"{:,}".format(int(standard*df2_num[i,1])).strip()+")"
            print(STRING.strip(),end="")
            result_out=result_out+STRING.strip()+"\n"



        PREDICT    ="None"
        #  売り予測の場合
        if PRE2[2]>0.5:
            PREDICT="sell"
            #print("売:",end=" ")
            profit_rate=df2_num[i,open_index]/df2_num[i, close_index]
            MONEY=(profit_rate)*MONEY

        #  買い予測の場合
        if PRE2[0]>0.5:
            PREDICT="buy"
            #print("買",end=" ")
            profit_rate=df2_num[i,close_index]/df2_num[i,open_index]
            MONEY=(profit_rate)*MONEY

        if df2_num[i,63]==1:
            Y_result="buy"
        elif df2_num[i,65]==1:
            Y_result="sell"
        else:
            Y_result="None"
            
        seigo="×"    
        if Y_result==PREDICT:
            seigo="●"
        
        if profit_rate!=0:
            profit_rate=100*(profit_rate-1)
        
        s1 = pd.Series([df2_num[i,0],                 #日付
                        round(df2_num[i,open_index], 2),        #OPEN
                        round(df2_num[i,close_index],2),        #Close
                        round((df2_num[i,close_index]/df2_num[i,open_index]-1)*100,2),  #アップ率
                        round(MONEY),
                        int(standard*df2_num[i,1]),                 #基準価格
                        Y_result,PREDICT,seigo,
                        round(profit_rate,2)],                     #予測
               index=df_sim.columns)
        df_sim=df_sim.append(s1,ignore_index=True)
        

In [ ]:
df_sim[(
    df_sim['Date'].str.contains('01-01') |
    df_sim['Date'].str.contains('04-01') |
    df_sim['Date'].str.contains('07-01') |
    df_sim['Date'].str.contains('10-01') 
)]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

df_sim.plot(x='Date',y='money',title='Money/Date')

In [ ]:
print("●",df_sim[df_sim['正誤']=="●"].shape)
print("X",df_sim[df_sim['正誤']=="×"].shape)

for i in range(7,19):
    YEAR=str(i+2000)
    start=YEAR+"-01-01"
    end=YEAR+"-12-31"

    tmp=df_sim.query("Date >= @start and Date <= @end ")
    
    print("{:}年の利益率{:.2f}%".format(YEAR,tmp[['利益率']].sum()[0]))

# 特徴選択

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

skb=SelectKBest(chi2,k=10)

skb.fit(trainX,trainY2)
trainX_new=skb.transform(trainX)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedKFold
model = SVC(kernel='linear', random_state=None)
k_range=np.arange(1,20)
scores=[]
std=[]

clf = XGBClassifier()
#clf = SVC(kernel='linear', random_state=None)



for k in k_range:
    ss= StratifiedKFold(n_splits  =5,
                       shuffle    =True,
                      random_state=2)
    score=[]
    
    for train_index, val_index in ss.split(trainX,trainY2):
        
        X_train2,X_val =trainX[train_index],trainX[val_index]
        y_train2,y_val =trainY2[train_index],trainY2[val_index]
        
        skb=SelectKBest(chi2,k=k)
        skb.fit(X_train2,y_train2)
        
        X_new_train2=skb.transform(X_train2)
        X_new_val   =skb.transform(X_val)
        
        clf.fit(X_new_train2,y_train2)
        score.append(clf.score(X_new_val,y_val))
    scores.append(np.array(score).mean())
    std.append(np.array(score).std())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(k_range,scores)
plt.errorbar(k_range,scores,yerr=std)
plt.ylabel("accuracy")
plt.xlim(0,20)

# どの説明変数が聞いているのか

In [ ]:
import warnings
warnings.filterwarnings('ignore')
k=5

clf = XGBClassifier()
Label=train.iloc[:,3:63]

skb=SelectKBest(chi2,k=k)
X=skb.fit(trainX,trainY2)

for i in range (skb.get_support().shape[0]):

    if skb.get_support()[i]== True:
        print(skb.get_support()[i],end=" ")
        print(Label.columns[int(i)])